In [3]:
import pymatgen
import numpy as np
import pandas as pd
import chemparse
from scipy.spatial import ConvexHull

fulldataset = pd.read_csv('fulldataset_v2.csv')
oxides = pd.read_csv('oxide_data.csv')

fulldataset = np.array(fulldataset)
periodictable = np.array(periodictable)
oxides = np.array(oxides)
processedoxides = []
processedoxides_els = []


FileNotFoundError: [Errno 2] No such file or directory: 'oxide_data.csv'

In [ ]:
for oxide in oxides:
    oxide_elements = np.sort(np.asarray(list(chemparse.parse_formula(oxide[1]).keys())))
    processedoxides.append(oxide)
    processedoxides_els.append(oxide_elements)
processedoxides = np.column_stack((processedoxides, processedoxides_els))

from scipy.optimize import minimize
# Using minimization 
def objective(a_init, matrix_composition, b_coeff, filtered_formEnet):
    # print(np.sum(a_coeff*formE))
    calc_coeff = b_coeff - np.dot(matrix_composition.T, a_init)
    distance = np.sqrt(np.sum(calc_coeff ** 2))
    
    # weight1 = 1/(distance+0.05)
    # weight2 = 0
    # for i in calc_coeff:
    #     if i > 0.3:
    #         weight2 = 1 / ((i+1)*100)
    positives = 0
    for i in a_init:
        if i < 0:
            positives += abs(i)
    #### FOR NELDER MEAD
    positivityweight = 1 / (20**positives)
    massweight = 1 / (8**distance)

    return np.dot(a_init,filtered_formEnet) * positivityweight * massweight

def preservepositivity(a_init):
    return a_init

# def preservepositivity_prod(a_init, matrix_composition, b_coeff):
#     calc_coeff = np.dot(matrix_composition.T,a_init) - b_coeff
#     return 0.03 - calc_coeff

def preservemass(a_init, matrix_composition, b_coeff):
    #### Ensure equality, so this type of constraint is 'type': 'eq'
    calc_coeff = b_coeff - np.dot(matrix_composition.T,a_init)
    return calc_coeff
    # if np.sum(np.abs(b_coeff - np.dot(matrix_composition.T,a_init))) < 0.1:
    #     return 0
    # else:
    #     return 1

def minimizedweights(material_name, filteredoxides):
    material_elements = np.concatenate((np.asarray(list(chemparse.parse_formula(material_name).keys())),['O'])).tolist()
    material_coeffs = np.concatenate((np.asarray(list(chemparse.parse_formula(material_name).values())),[0]))
    
    # filteredoxides_formEperatom = filteredoxides[:,2]
    filtered_formEnet = []
    matrix_composition = np.empty((0, len(material_elements)))
    # matrix_composition = np.zeros((len(material_elements), len(filteredoxides)))
    
    for filteredoxide in filteredoxides:
        filteredoxide_name= np.asarray(list(chemparse.parse_formula(filteredoxide[1]).keys()))
        filteredoxide_coefficients = np.asarray(list(chemparse.parse_formula(filteredoxide[1]).values()))
        filteredoxide_arr = np.vstack((filteredoxide_name,filteredoxide_coefficients))
        numatoms = np.sum(filteredoxide_coefficients)
        formEnet = numatoms * filteredoxide[2]
        filtered_formEnet.append(formEnet)
        ### Now fill in matrix_comp
        temp_row = [0] * len(material_elements)
        for letters, values in zip(filteredoxide_arr[0], filteredoxide_arr[1]):
            index = material_elements.index(letters)
            temp_row[index] = values
        matrix_composition = np.vstack((matrix_composition, temp_row))
    
    matrix_composition = np.asarray(matrix_composition, dtype=float)
    filtered_formEnet = np.asarray(filtered_formEnet, dtype=float)
    
    ### TOGGLE Find convex hull
    perturbation_comp = np.random.randn(*matrix_composition.shape) * 1e-6
    perturbation_form = np.random.randn(*filtered_formEnet.shape) * 1e-6
    matrix_composition = matrix_composition + perturbation_comp
    filtered_formEnet = filtered_formEnet + perturbation_form

    data_points = np.column_stack((matrix_composition, filtered_formEnet))
    if len(data_points) > 4:
        hull = ConvexHull(data_points)
        matrix_composition = matrix_composition[hull.vertices]
        filtered_formEnet = filtered_formEnet[hull.vertices]
        filteredoxides = filteredoxides[hull.vertices]
        
    
    b_coeff = np.asarray(material_coeffs, dtype=float)
    pseudo_inv = np.linalg.pinv(matrix_composition.T)
    
    
    ### Try different initial points
    # TOGGLE Naive way
    # a_init = np.zeros(len(filtered_formEnet))
    # indexgreatest = np.argmax(filtered_formEnet)
    # a_init[indexgreatest] = 1
    
    #Calculate inverse
    a_init = np.dot(pseudo_inv, b_coeff)
    
    ## TOGGLE For Nelder-Mead disable constraints
    # mass_constraint = {'type': 'eq', 'fun': preservemass, 'args': (matrix_composition, b_coeff)}
    # positivity_constraint ={'type': 'ineq', 'fun': preservepositivity}
    # positivity_constraintprod ={'type': 'eq', 'fun': preservemass, 'args': (matrix_composition, b_coeff)}
    result = minimize(objective, a_init, args=(matrix_composition, b_coeff, filtered_formEnet),
                        method = 'Nelder-Mead',options = {'maxiter': 50000})
    # result = minimize(objective, a_init, args=(matrix_composition, b_coeff, filtered_formEnet), constraints= [positivity_constraint, mass_constraint],
    #                   method = 'Nelder-Mead',options = {'maxiter': 50000})
    
    a_final = result.x
    netE = np.dot(a_final, filtered_formEnet)
    
    return netE, a_final, material_elements, matrix_composition, filteredoxides
        

def getoxides(material_name):
    filteredoxides = []
    material_elements = np.concatenate((np.sort(np.asarray(list(chemparse.parse_formula(material_name).keys()))), ['O']))
    for oxide in processedoxides:
        if all(elem in material_elements for elem in oxide[-1]):
            filteredoxides.append(oxide)
    filteredoxides = np.array(filteredoxides)
    unique_rows, unique_indices = np.unique(filteredoxides[:,1], return_index=True)
    unique_reloxides = filteredoxides[unique_indices]

    ### Remove low formation energy oxides
    
    return unique_reloxides

def oxidemetriccalculator(dset, indexstart, indexend):
    arr_formEs = []
    arr_coeffs = []
    arr_matels = []
    arr_matrixcomp = []
    arr_oxidenames = []
    for material in dset[indexstart:indexend]:
        name = material[5]
        relevantoxides = getoxides(name)
        formEs, coeffs, material_elements, matrix_comp, filteredoxides = minimizedweights(name, relevantoxides)
        arr_formEs.append(formEs)
        arr_coeffs.append(coeffs)
        arr_matels.append(material_elements)
        arr_matrixcomp.append(matrix_comp)
        arr_oxidenames.append(filteredoxides[:,1])
        
    arr_formEs = pd.DataFrame(arr_formEs)
    arr_coeffs = pd.DataFrame(arr_coeffs)
    arr_matels = pd.DataFrame(arr_matels)
    arr_matrixcomp = pd.DataFrame(arr_matrixcomp)
    arr_oxidenames = pd.DataFrame(arr_oxidenames)
    
    return arr_formEs, arr_coeffs, arr_matels, arr_matrixcomp, arr_oxidenames

arr_formEs_06, arr_coeffs_06, arr_matels_06, arr_matrixcomp_06, arr_oxidenames_06 = oxidemetriccalculator(fulldataset,150000,175982)
arr_formEs_06.to_csv('06_refinedoxidemetric_alloxides_formE.csv',header=False, index=False)
arr_coeffs_06.to_csv('06_refinedoxidemetric_alloxides_arr_coeffs.csv', header=False, index=False)
arr_matels_06.to_csv('06_refinedoxidemetric_alloxides_arr_matels.csv', header=False, index=False)
arr_matrixcomp_06.to_csv('06_refinedoxidemetric_alloxides_arr_matrixcomp.csv', header=False, index=False)
arr_oxidenames_06.to_csv('06_refinedoxidemetric_alloxides_arr_oxidenames.csv', header=False, index=False)